In [1]:
# common python modules
import sys
import math
#import warnings
#warnings.filterwarnings("ignore")


# EDA and data visualization modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Modules for regression Analysis
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Normalizer
from sklearn import metrics

In [2]:
data = pd.read_csv('data_files/fifa21_male2.csv')
data.columns = [i.replace(' ', '_').lower() for i in data.columns]

/Users/sebastianvier/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Reusable functions

def slice_by_values(columns, init, end):
    init = list(columns).index(init)
    end = list(columns).index(end)
    return list(columns[init:end + 1])

def remove_sign(val, sign):
        if (type(val) == str) and (val[0] == sign):
            return val[1:]
        else:
            return val

def devide_position_generator(df, both=False):
    def devide_position_columns(value, pos):
        array =  value.split('+')
        return int(array[pos].strip())
    
    for column in df.columns:
        yield df[column].apply(lambda x : devide_position_columns(x,0))
        if both:
            yield df[column].apply(lambda x : devide_position_columns(x,1))
            
def change_first_val(arr, f_value):
    lst = [f_value]
    for value in arr:
        if value != f_value:
            lst.append(value)
    return lst

def add_monetary_2_numeric(monetary_df):
    for column in monetary_df.columns:
        if type(monetary_df[column].iloc[0]) == str:
            monetary_df[column].iloc[0][0]
            col_first_symbol = monetary_df[column].iloc[0][0] #of each row of each column
            if col_first_symbol == '€':
                yield pd.to_numeric(monetary_df[column].apply(lambda x : euros2numbers(x)))

def euros2numbers(val):
    """Turn values with € sign (€20K) into float values for math calculations"""
     
    def convert2number(val):
        if type(val) == str:
            if val[-1] == 'K':
                return float(val[:-1]) * (10 ** 3)
            if val[-1] == 'M':
                return float(val[:-1]) * (10 ** 6)
            else:
                return val
        else:
            return val 
    val = remove_sign(val, '€')
    val = convert2number(val)
    return val
            
def add_position_2_numeric(categorical_df):
    columns = slice_by_values(categorical_df.columns,init='ls',end='gk')
    
    transformed_df = pd.concat([i for i in devide_position_generator(categorical_df[columns])], axis=1)
    result = pd.concat([numerical, transformed_df], axis=1)
    return result, columns


def filter_condition_3(value, threshold=.1):
    if value[-1] != 'K':
        return False
    if int(value[1:-1]) > (escudero_wage + escudero_wage*threshold):
        return False
    if int(value[1:-1]) < (escudero_wage - escudero_wage*threshold):
        return False
    else:
        return True
    
def eliminate_collinearity(data, threshold=.1):
    def add_values_2_drop(df, threshold):
        for i in range(1, len(df)):
            if df[i] > (1-threshold):
#                 print(f'Correlation found between {df.index[0]} and {df.index[i]}')
                drop_list.append(df.index[i])

    def drop_columns_from_init():
        for column in initial_columns:
            if column in drop_list:
                initial_columns.remove(column)
            
    drop_list = []
    checked_columns = []
    initial_columns = list(data.columns)
    corr = data[initial_columns].corr().iloc[0,:] 


    while len(initial_columns) > 2:
        add_values_2_drop(corr, threshold)
        checked_columns.append(initial_columns[0])
        initial_columns = initial_columns[1:]
        drop_columns_from_init()
        corr = data[initial_columns].corr().iloc[0,:]
    result = drop_list
    return drop_list

def uncorrelated_col(correlation, threshold =.6):
    drop_list = []
    corr_column = correlation.iloc[0]
    for i in range(len(corr_column)):
        if corr_column[i] < threshold:
            drop_list.append(corr_column.index[i])
    return drop_list
            
    
def get_r2_score(data, column):
    pass
    data = numerical
    X = data[[column, column]]
    y =data.release_clause

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=35)

    ln = LinearRegression().fit(X_train, y_train)
    prediction = ln.predict(X_test)

    r2_score = metrics.r2_score(y_test, prediction)
    mse = metrics.mean_squared_error(y_test, prediction)
    rmse = np.sqrt(mse)
    return r2_score

### Some information about the data

In [4]:
# How many teams have 20 or more players?
print('Number of teams with more than 20 players:')
num_teams_20 = len(data.iloc[:,5].value_counts()[data.iloc[:,5].value_counts() >= 20])
print(num_teams_20)

# Number of total teams
print('\nTotal number of teams')
num_teams = data.iloc[:,5].nunique()
print(num_teams)

Number of teams with more than 20 players:
446

Total number of teams
917


In [5]:
# Percentage of teams with less 20 players or more
print('\nPercentage of teams with 20 players or more')
print(round(num_teams_20/num_teams,2))


Percentage of teams with 20 players or more
0.49


##### This means that more than half of the teams in the data set have less than 20 players. Probably there is some players missing

### Decide the best replacement for a transfered player in Sevilla FC

 First we are going to decide what player is going to be transfered.
 We select a player who is leaving on 2021 and has the highest wage.
A player with a high wage is likely a player who is playing most matches.

In [6]:
sevilla_fc = data[data['club'] == 'Sevilla FC']
sorted_index = sevilla_fc['wage'].map(lambda x : int(x[1:-1])).sort_values(ascending=False).index
## This codes strip the K and € sing gets the index to sort the players
sevilla_fc.loc[sorted_index[:10]][['name', 'nationality', 'contract', 'wage', 'age']]
## That index is used then on the sevilla_fc to sort the playes by wage.
## The [:10] is used to only get the first 10 players.

,name,nationality,contract,wage,age
569,Jesús Navas,Spain,2017 ~ 2021,€43K,34
7631,M. Acuña,Argentina,2020 ~ 2024,€41K,28
4593,L. Ocampos,Argentina,2019 ~ 2024,€40K,25
6702,Diego Carlos,Brazil,2019 ~ 2024,€39K,27
4048,Suso,Spain,2020 ~ 2025,€39K,26
1061,I. Rakitić,Croatia,2020 ~ 2024,€39K,32
1925,Fernando,Brazil,2019 ~ 2022,€39K,32
2482,L. de Jong,Netherlands,2019 ~ 2023,€38K,29
2845,Escudero,Spain,2015 ~ 2021,€31K,30
7624,Joan Jordán,Spain,2019 ~ 2023,€30K,25


So the player the players who is leaving is **Sergio Escudero**. Actually he is not playing in Sevilla anymore as to 2021.

### Candidates

 ###### First we check the players that have the same best position and the best position of Escudero

In [7]:
bp_escudero = data.iloc[2845]['bp']
position_escudero = data.iloc[2845]['position']
## This is the easy way to transfomr the wage into the numeric value
escudero_wage = int(data.iloc[2845]['wage'][1:-1])

## Amount of players that have exactly the same position

## This is to reduce the data I am currently using.
info_data = data[slice_by_values(data.columns, 'id', 'contract')]

# Same position and same best position
condition_1 = info_data['position'] == position_escudero
condition_2 = info_data['bp'] == bp_escudero

###### Now we check at a player with a similar Wage and whos age is no more than 33

In [8]:
# Around the same wage condition_3
    
condition_3 = info_data['wage'].apply(lambda x : filter_condition_3(x, .40)) 
condition_4 = info_data['age'] < 33

## Create a global condition
conditions = condition_1 & condition_2 & condition_3 & condition_4
candidates = data[conditions]

In [9]:
candidates.shape

(14, 107)

In [10]:
candidates

,id,name,age,ova,nationality,club,bov,bp,position,player_photo,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,gender
1001,167305,J. Vargas,31,76,Peru,Real Betis,74,LB,LB LM,https://cdn.sofifa.com/players/167/305/17_120.png,...,73+1,73+1,74+1,74+1,73+1,73+1,73+1,74+1,16+1,Male
2072,186347,M. Rybus,30,74,Poland,Lokomotiv Moscow,74,LB,LB LM,https://cdn.sofifa.com/players/186/347/20_120.png,...,71+2,71+2,73+1,72+2,69+2,69+2,69+2,72+2,19+2,Male
2705,191390,F. N'Sakala,29,72,DR Congo,Beşiktaş JK,72,LB,LB LM,https://cdn.sofifa.com/players/191/390/20_120.png,...,68+2,68+2,71+1,70+2,67+2,67+2,67+2,70+2,17+2,Male
2845,192679,Escudero,30,80,Spain,Sevilla FC,80,LB,LB LM,https://cdn.sofifa.com/players/192/679/20_120.png,...,77+2,77+2,77+2,78+2,77+2,77+2,77+2,78+2,20+2,Male
3873,201530,B. Martella,27,71,Italy,Brescia,71,LB,LB LM,https://cdn.sofifa.com/players/201/530/20_120.png,...,67+2,67+2,70+1,69+2,64+2,64+2,64+2,69+2,15+2,Male
4420,204757,Toño García,30,76,Spain,Levante UD,76,LB,LB LM,https://cdn.sofifa.com/players/204/757/20_120.png,...,72+2,72+2,75+1,74+2,68+2,68+2,68+2,74+2,20+2,Male
4882,207924,Lucas Lima,28,75,Brazil,Al Ahli,75,LB,LB LM,https://cdn.sofifa.com/players/207/924/20_120.png,...,73+2,73+2,75+0,73+2,69+2,69+2,69+2,73+2,18+2,Male
5626,212187,P. Max,26,79,Germany,PSV,79,LB,LB LM,https://cdn.sofifa.com/players/212/187/20_120.png,...,74+2,74+2,78+2,77+2,71+2,71+2,71+2,77+2,19+2,Male
5850,213317,Kim Jin Su,28,71,Korea Republic,Al Nassr,71,LB,LB LM,https://cdn.sofifa.com/players/213/317/20_120.png,...,66+2,66+2,69+2,69+2,66+2,66+2,66+2,69+2,15+2,Male
6010,214026,J. Mojica,27,75,Colombia,Atalanta,75,LB,LB LM,https://cdn.sofifa.com/players/214/026/20_120.png,...,71+2,71+2,74+1,73+2,70+2,70+2,70+2,73+2,18+2,Male


### What do the best teams say?

 Our idea is to check how the best teams in the world decide on their LB and ther RB and try to emulate their choices.

##### Finding the best teams

For this we are going to use a group by on value.

In [11]:
best_teams = data[['name','club']]
monetary = data[['value']]
monetary = pd.concat([i for i in add_monetary_2_numeric(monetary)], axis=1)
best_teams = pd.concat([best_teams, monetary], axis=1)
best_teams = best_teams.groupby('club').sum().sort_values('value', ascending=False).head(21)
best_teams = list(best_teams[best_teams.index != 'Sevilla FC' ].index)

In [12]:
condition_1 = data.club.apply(lambda x: x in best_teams)
condition_2 = data.bp.apply(lambda x: x in ['LB', 'RB'])
model_data = data[(condition_1) & (condition_2)]

### Cleaning the data

In [13]:
## Delete all unnecesary columns
unnesary_columns = ['player_photo', 'club_logo', 'flag_photo', 'team_&_contract', 'loan_date_end']
numerical = model_data.select_dtypes(np.number).drop('id', axis=1)
categorical = model_data.select_dtypes(exclude=np.number).drop(unnesary_columns,axis=1, errors='ignore')

In [14]:
## Transform position in the court to the numeric values and drop them from categorical
numerical, droping_columns = add_position_2_numeric(categorical)
categorical = categorical.drop(droping_columns, axis=1,errors='ignore')

In [15]:
## Change the values concerning money and add them to numerical
monetary = pd.concat([i for i in add_monetary_2_numeric(categorical)], axis=1)
numerical = pd.concat([numerical, monetary], axis=1)

### Eliminating collinearity in the data

In [16]:
numerical.shape

(78, 84)

In [17]:
drop_list = eliminate_collinearity(numerical, threshold=.03)
numerical = numerical.drop(drop_list, axis=1)

## Add the release clause because this is going to be our target value
numerical = pd.concat([monetary.release_clause,numerical], axis=1)

## I don't want to have value because I prefer to use release_clause which is decided by the teams
numerical = numerical.drop(['age''value'], axis=1, errors='ignore')

In [18]:
numerical.shape

(78, 52)

### Regression Models

Base Model

In [19]:
## Drop 0 values
numerical = numerical[numerical.release_clause != 0]
X = numerical.drop('release_clause', axis=1)
y = numerical.release_clause
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=35)


ln = LinearRegression().fit(X_train, y_train)
prediction = ln.predict(X_test)

In [20]:
r2_score = metrics.r2_score(y_test, prediction)
mse = metrics.mean_squared_error(y_test, prediction)
rmse = np.sqrt(mse)

In [21]:
r2_score

0.9432760382478903

Model_2

In [23]:
correlation = numerical.corr()
uncorrelated_list = uncorrelated_col(correlation, .6)
data = numerical.drop(uncorrelated_list, axis=1)
    
X = data.drop('release_clause', axis=1)
y =data.release_clause

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=35)

ln = LinearRegression().fit(X_train, y_train)
prediction = ln.predict(X_test)

r2_score = metrics.r2_score(y_test, prediction)
mse = metrics.mean_squared_error(y_test, prediction)
rmse = np.sqrt(mse)

In [24]:
r2_score

0.9664246704916205

Ok the model is working but probably this is because I am taking into accoun the **wage** .

In [25]:
data = numerical
X = data[['value', 'value']]
y =data.release_clause

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=35)

ln = LinearRegression().fit(X_train, y_train)
prediction = ln.predict(X_test)

r2_score = metrics.r2_score(y_test, prediction)
mse = metrics.mean_squared_error(y_test, prediction)
rmse = np.sqrt(mse)
print(r2_score)

0.9641718465159181


But actually not. The wage is not the factors that rises the r2_score, it is actually value

In [26]:
drop_list = uncorrelated_col(correlation, .6)
drop_list.append('value')
drop_list.append('wage')
data = numerical.drop(drop_list, axis=1)
    
X = data.drop('release_clause', axis=1)
y =data.release_clause

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=35)

ln = LinearRegression().fit(X_train, y_train)
prediction = ln.predict(X_test)

r2_score = metrics.r2_score(y_test, prediction)
mse = metrics.mean_squared_error(y_test, prediction)
rmse = np.sqrt(mse)

In [27]:
r2_score

0.36709632000823533

But of course I need to to drop value and wage. And so we get a poor 36%.

But we can still change the threshold for droping columns

In [28]:
drop_list = uncorrelated_col(correlation, .70)
drop_list.append('value')
drop_list.append('wage')
data = numerical.drop(drop_list, axis=1)
    
X = data.drop('release_clause', axis=1)
y =data.release_clause

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=35)

ln = LinearRegression().fit(X_train, y_train)
prediction = ln.predict(X_test)

r2_score = metrics.r2_score(y_test, prediction)
mse = metrics.mean_squared_error(y_test, prediction)
rmse = np.sqrt(mse)

In [29]:
r2_score

0.7272946548671229

In [30]:
drop_list = uncorrelated_col(correlation, .70)
drop_list.append('value')
drop_list.append('wage')
data = numerical.drop(drop_list, axis=1)
    
X = data.drop('release_clause', axis=1)
y =data.release_clause

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=35)

ln = LinearRegression().fit(X_train, y_train)
prediction = ln.predict(X_test)

r2_score = metrics.r2_score(y_test, prediction)
mse = metrics.mean_squared_error(y_test, prediction)
rmse = np.sqrt(mse)
print(r2_score)

0.7272946548671229


And 72% is the best we can get.<br>
Lets check what does columns are

In [31]:
list(X.columns)

['ova', 'pot', 'stamina', 'base_stats']

We were expecting 'ova' and 'pot', but we also got stamina and base_stats. What you are looking for is a player that is good in many areas, but not oustanding. And that can mantain his pace during the whole game.

Lets see what are this players like in the best 3 teams of the world.

In [32]:
data = pd.read_csv('data_files/fifa21_male2.csv')
data.columns = [i.replace(' ', '_').lower() for i in data.columns]

best_teams = data[['name','club']]
monetary = data[['value']]
monetary = pd.concat([i for i in add_monetary_2_numeric(monetary)], axis=1)
best_teams = pd.concat([best_teams, monetary], axis=1)
best_teams = best_teams.groupby('club').sum().sort_values('value', ascending=False).head(3)

/Users/sebastianvier/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [33]:
best_teams = best_teams.index
condition_1 = data.club.apply(lambda x: x in best_teams)
condition_2 = data.bp.apply(lambda x: x in ['LB','RB'])
best_players = data[(condition_1) & (condition_2)]

In [34]:
best_players[['name','age','club','bp','bov', 'pot', 'stamina', 'base_stats']]

,name,age,club,bp,bov,pot,stamina,base_stats
572,Sylvinho,35,Manchester City,LB,78,84,67,431
1382,Marcelo,32,Real Madrid,LB,83,83,74,472
2264,K. Walker,30,Manchester City,RB,85,85,89,470
2556,N. Clyne,28,Liverpool,RB,76,76,64,419
4462,Carvajal,28,Real Madrid,RB,86,86,89,454
6387,A. Robertson,26,Liverpool,LB,87,89,95,461
8464,F. Mendy,25,Real Madrid,LB,83,88,88,471
9212,T. Alexander-Arnold,21,Liverpool,RB,87,92,88,464
9867,Odriozola,24,Real Madrid,RB,79,82,79,418
14984,Y. Larouci,19,Liverpool,LB,61,80,60,352


In [35]:
r2_bov = get_r2_score(data, 'ova')
r2_pot = get_r2_score(data, 'pot')
r2_stamina = get_r2_score(data, 'stamina')
r2_base_stats = get_r2_score(data, 'base_stats')

In [36]:
def yield_columns():
    yield r2_bov * candidates[['bov']]
    yield r2_pot * candidates[['pot']]
    yield r2_stamina * candidates[['stamina']]
    yield r2_base_stats * candidates[['base_stats']]

In [37]:
transformed_columns = pd.concat(yield_columns(), axis=1)
r2_calculation_sum = transformed_columns.sum(axis=1)
candidates_final = candidates[['name', 'bp','nationality', 'club','age']]
candidates_final = pd.concat([candidates_final, r2_calculation_sum], axis=1)
candidates_final.columns = ['name', 'bp', 'nationality', 'club','age', 'calculation_sum']

In [38]:
monetary = candidates[['value', 'wage']]
numerical_monetary = pd.concat([i for i in add_monetary_2_numeric(monetary)],axis=1)
candidates_final = pd.concat([candidates_final, numerical_monetary], axis=1)

In [39]:
candidates_final

,name,bp,nationality,club,age,calculation_sum,value,wage
1001,J. Vargas,LB,Peru,Real Betis,31,331.507306,2300000.0,20000.0
2072,M. Rybus,LB,Poland,Lokomotiv Moscow,30,317.853211,4400000.0,35000.0
2705,F. N'Sakala,LB,DR Congo,Beşiktaş JK,29,310.322200,2600000.0,38000.0
2845,Escudero,LB,Spain,Sevilla FC,30,328.437142,11500000.0,31000.0
3873,B. Martella,LB,Italy,Brescia,27,308.102439,2100000.0,20000.0
4420,Toño García,LB,Spain,Levante UD,30,323.559410,6000000.0,21000.0
4882,Lucas Lima,LB,Brazil,Al Ahli,28,319.556127,5500000.0,33000.0
5626,P. Max,LB,Germany,PSV,26,337.400014,13000000.0,20000.0
5850,Kim Jin Su,LB,Korea Republic,Al Nassr,28,295.431243,2000000.0,22000.0
6010,J. Mojica,LB,Colombia,Atalanta,27,323.588406,6000000.0,40000.0


In [40]:
candidates_final['best_value'] = candidates_final['calculation_sum'] / candidates_final['value']

In [41]:
final_candidates_index = candidates_final.sort_values('best_value', ascending=False).head().index

In [42]:
final_candidates = data.loc[final_candidates_index]

In [43]:
final_candidates[['name','age','club','bp','bov', 'pot', 'stamina', 'base_stats', 'contract', 'value', 'wage']]

,name,age,club,bp,bov,pot,stamina,base_stats,contract,value,wage
5850,Kim Jin Su,28,Al Nassr,LB,71,71,72,387,2020 ~ 2025,€2M,€22K
3873,B. Martella,27,Brescia,LB,71,71,86,402,2019 ~ 2022,€2.1M,€20K
1001,J. Vargas,31,Real Betis,LB,74,76,73,456,2015 ~ 2017,€2.3M,€20K
2705,F. N'Sakala,29,Beşiktaş JK,LB,72,72,83,407,2020 ~ 2021,€2.6M,€38K
8875,Alam Araújo,28,Grêmio,LB,72,72,70,396,2019 ~ 2023,€2.7M,€19K


### Conclusion